Between May 24th and May 28th [Anastasios (Tasos) Sidiropoulos](https://sidiropo.people.uic.edu/) provided several counter examples indicating that [our constraints for the the polytope of wealth distributions](https://github.com/renepickhardt/Lightning-Network-Limitations/blob/906b3a4d691cbfb30c2f385c6697b4fe2bb81676/Limits%20of%20two%20party%20channels/Geometric%20Argument%20for%20Bimodal%20Liquidity%20Distribution%20in%20Payment%20Channels%20of%20the%20Lightning%20Network.ipynb) are too weak. In particular he provided [a point in our polytope for which no feasible lighting network state existed](https://twitter.com/sidiropo/status/1795458687500239055)

His example assumes the following network

```
(A)---2---(B)---2---(C)---2---(D)---2---(E)---2---(F)
```
where `A,B,C,F` hold `1` coin each and `D,E` hold `3`coins. 

one can easily check that $w_i+w_j>=c_{ij}$ and no user has more coins than total capacity of the neighboring channels. 

However no valid state for this network exists that produces this particular wealth distribution. This means our polytope of wealth distributions is too large and the formulation of inequalities cannot hold. 

## Smaller example

The example of Sidiropoulos can actually be made smaller. Consider the following network: 
```
(A)---2---(B)---2---(C)---2---(D)---2---(E)
```
where `A,B,C` hold `1` coin each and `D` holds `3` coins and `E` holds `2` coins. 

Again no network state exists that would produce this wealth distribution. 

## Goal of this notebook

Leaving the [formulation of inequalities to describe the polytope of wealth distributions]((https://github.com/renepickhardt/Lightning-Network-Limitations/blob/906b3a4d691cbfb30c2f385c6697b4fe2bb81676/Limits%20of%20two%20party%20channels/Geometric%20Argument%20for%20Bimodal%20Liquidity%20Distribution%20in%20Payment%20Channels%20of%20the%20Lightning%20Network.ipynb)) and focussing on the polytope $S$ of channel states and the natural mapping from $\omega: S\rightarrow S/\sim \cong W$ one is interested if for $w\in S/\sim$ we can construct and element of $\omega^{-1}(\{w\})$ (aka an element in the preimage of $w$)

We claim that a preimage exists if the following system of linear equations has a non trivial solution. This can be tested by solving the linear program constructed and provided in this notebook. The system of linear equations is described in the code! but the gist is to have 
* $m$ constraints with conservation of liquidity in channels
* $n$ constraints relating to the wealth distribution of $n$ peers
* $1$ constraint relating to the total number of coins
* $2m$ variables encoding the wealth of each user in the channel (this can probably be reduced to $m$ variables)

## Some numbers about constraints and variables

* $m$ number of edges
* $2m$ variables ($2$ for each channel) aka number of collums
* $n$ number of nodes
* $m+n+1$ constraints aka number of rows
    * $m$ constraints for each channel
    * $n$ contraints as the wealth for each node
    * $1$ constraint as the number of coins is fixed
* $2m+1$ bounds actually $m$ bounds but each is twice and 1 bound for total number of coins

## Acknowledgements

Obviously a big thanks goes to [Anastasios (Tasos) Sidiropoulos](https://sidiropo.people.uic.edu/) for the initial counter example and the discussion on twitter. Also to Stefan Richter for putting out the hypothesis in the thread that a cycle base would be sufficient to test for conservation of liquidity. The work is sponsored through a [grant from OpenSats](https://opensats.org/blog/rene-pickhardt-receives-lts-grant) and through [individual patreons](https://www.patreon.com/renepickhardt).

In [1]:
from scipy.optimize import linprog


Create channels and wealth distribution according to the modified counter example from Sidiropoulos

In [2]:
channels = {(0,1):2, (1,2):2, (2,3):2, (3,4):2}
wealth = {0:1,1:1,2:1,3:3,4:2}
# Comment in the following to have a feasible wealth distribution
#wealth = {0:1,1:2,2:1,3:2,4:2}

Ad-hoc test to see that the original wealth polytope inequalities are held in this counterexample

In [3]:
flag = True
for chan, cap in channels.items():
    s,d = chan
    if wealth[s]+wealth[d] < cap:
        print("no")
        flag = False
for u,w in wealth.items():
    if u == 0 or u == 4:
        if w > 2:
            print("no")
            flag = False

    else:
        if w > 4:
            print("no") 
            flag = False

if flag:
    print("polytope inequalities satisfied")

polytope inequalities satisfied


Construction of the system of linear equations and the corresponding linear program that must be feasible in order for a lighting network channel state to exist that produces the wealth distribution. 

a feasible solution means that we have a non trivial preimage for a given wealth distribution

In [4]:
num_coins = sum(channels.values())
m = len(channels)
n = len(wealth)

channel_idx = {}
A = []
b = []

wealth_constraints = []
for i in range(n):
    row = [0]*2*m
    wealth_constraints.append(row)

bounds = []
for k, chan in enumerate(channels.items()):
    row = [0]*2*m
    row[2*k]=1
    row[2*k+1]=1
    A.append(row)
    key,cap = chan
    #add bounds of channel twice as each channel adds two variables
    bounds.append((0,cap))
    bounds.append((0,cap))

    x,y = key
    channel_idx[2*k]=key
    channel_idx[2*k+1]=(y,x)
    
    wealth_constraints[x][2*k]=1
    wealth_constraints[y][2*k+1]=1
    b.append(cap)
for k, row in enumerate(wealth_constraints):
    A.append(row)
    b.append(wealth[k])
A.append([1]*2*m)
num_coins=sum(wealth.values())
b.append(num_coins)

c = [1]*2*m

A,b,bounds

([[1, 1, 0, 0, 0, 0, 0, 0],
  [0, 0, 1, 1, 0, 0, 0, 0],
  [0, 0, 0, 0, 1, 1, 0, 0],
  [0, 0, 0, 0, 0, 0, 1, 1],
  [1, 0, 0, 0, 0, 0, 0, 0],
  [0, 1, 1, 0, 0, 0, 0, 0],
  [0, 0, 0, 1, 1, 0, 0, 0],
  [0, 0, 0, 0, 0, 1, 1, 0],
  [0, 0, 0, 0, 0, 0, 0, 1],
  [1, 1, 1, 1, 1, 1, 1, 1]],
 [2, 2, 2, 2, 1, 1, 1, 3, 2, 8],
 [(0, 2), (0, 2), (0, 2), (0, 2), (0, 2), (0, 2), (0, 2), (0, 2)])

In [5]:
res = linprog(c, A_eq=A, b_eq=b, bounds=bounds)
if res.success == True:
    print("this state vector satisfies the wealth distribution")
    for i,x in enumerate(res.x):
        print(channel_idx[i], x)
else:
    print("The wealth distribution {} is infeasible for the network {}".format(wealth, channels))

The wealth distribution {0: 1, 1: 1, 2: 1, 3: 3, 4: 2} is infeasible for the network {(0, 1): 2, (1, 2): 2, (2, 3): 2, (3, 4): 2}


## Old example to play around and test the API

In [6]:
num_coins = 15
c = [1, 1, 1, 1, 1, 1]
dimensions = "A,B B,A A,C C,A B,C C,B".split(" ")
##### AB BA AC CA BC CB
    #### channel constraints
A = [[1,1,0,0,0,0], #AB + BA = cap 
     [0,0,1,1,0,0], #AC + CA = cap
     [0,0,0,0,1,1], #BC + CB = cap
     ###### wealth equations
     [1,0,1,0,0,0], #AB + AC = W_A
     [0,1,0,0,1,0], #BA + BC = W_B
     [0,0,0,1,0,1], #CA + CB = W_C
     [1,1,1,1,1,1]] # sum wealth = #coins
b = [2, 2, 11, 3, 5, 7, num_coins]
bounds = (0, num_coins)
res = linprog(c, A_eq=A, b_eq=b, bounds=[bounds for i in range(6)])
if res.success == True:
    print("this state vector satisfies the wealth distribution")
    for i,x in enumerate(res.x):
        print(dimensions[i], x)



this state vector satisfies the wealth distribution
A,B 1.0
B,A 1.0
A,C 2.0
C,A 0.0
B,C 4.0
C,B 7.0
